In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
times = np.array([6,8,10,12,14,17,21]).astype(int)

In [ ]:
%%time
adatas_pk = []
for ti in times:
# for ti in [6]:
    print(ti)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk.obs = adata_pk.obs.set_index('cell')
    print(adata_pk.var['peak'][:3].values)
    
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
adata.obs['cond_order'].unique()

In [ ]:
# get cells based on RNA
# get matrix based on ATAC - CPM or CP10k

qs_avc = []
l2fc_avc = []
# for cond_order, obssub in adata.obs.groupby('cond_order'):
for cond_order in range(7):
    obssub = adata[adata.obs['cond_order']==cond_order].obs
    
    # get sub
    adatasub = adata[obssub.index]
    adata_pk = adatas_pk[cond_order]
    
    # get A vs C 
    x = adatasub.obsm['pca_p17on'][...,0]
    cond_a = x < np.percentile(x,20)
    cond_c = x > np.percentile(x,80)
    cells_a = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_a].obs.index.values])
    cells_c = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_c].obs.index.values])
    
    cells_rna = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub.obs.index.values])
    adatasub.obs.index = cells_rna
    
    cells_atac = adata_pk.obs.index.values
    cells_mult = np.intersect1d(cells_rna, cells_atac) 
    
    adata_pk_a = adata_pk[np.intersect1d(cells_a, cells_atac)]
    adata_pk_c = adata_pk[np.intersect1d(cells_c, cells_atac)]
    
    mat_a = np.log10(1+adata_pk_a.X[...]) # layers['lognorm'][...]
    mat_c = np.log10(1+adata_pk_c.X[...]) # adatasub_c.layers['lognorm'][...]
    
    # DEGs
    lfc = np.log2(10)*(np.mean(mat_c, axis=0) - np.mean(mat_a, axis=0)) # log2FC (log10CPM as raw counts)
    ts, ps = stats.ttest_ind(mat_a, mat_c)
    rs, qs, _, _ = multipletests(np.nan_to_num(ps, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    
    num_sig = np.sum(np.logical_and(qs < 0.05, np.abs(lfc) > 1))
    print(cond_order, mat_a.shape, mat_c.shape, num_sig)
    
    # save this
    qs_avc.append(qs)
    l2fc_avc.append(lfc)
    
    # break
    
qs_avc = np.array(qs_avc)
l2fc_avc = np.array(l2fc_avc)

In [ ]:
l2fc_th = 1
alpha_th = 0.05
cond_sig   = np.logical_and(qs_avc < alpha_th, np.abs(l2fc_avc) > l2fc_th)
cond_sig_c = np.logical_and(qs_avc < alpha_th, l2fc_avc >  l2fc_th)
cond_sig_a = np.logical_and(qs_avc < alpha_th, l2fc_avc < -l2fc_th)

print('num AvsC-DEGs per cond:\t', cond_sig.sum(axis=1))
print('num A-DEGs per cond:\t',    cond_sig_a.sum(axis=1))
print('num C-DEGs per cond:\t',    cond_sig_c.sum(axis=1))

In [ ]:
cond_sig_c.shape

In [ ]:

fig, axs = plt.subplots(1,7, figsize=(4*7,4), sharex=True, sharey=True)
for cond_order in range(7):
    ax = axs[cond_order]
    time = times[cond_order]
    obssub = adata[adata.obs['cond_order']==cond_order].obs
    
    # get sub
    adatasub = adata[obssub.index]
    adata_pk = adatas_pk[cond_order]
    
    cells_rna = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub.obs.index.values])
    adatasub.obs.index = cells_rna
    cells_atac = adata_pk.obs.index.values
    cells_mult = np.intersect1d(cells_rna, cells_atac) 
    
    x = adatasub[cells_mult].obsm['pca_p17on'][...,0]
    adata_pk_a = adata_pk[:,cond_sig_a[cond_order]][cells_mult]
    adata_pk_c = adata_pk[:,cond_sig_c[cond_order]][cells_mult]
    mat_pk_a = np.log10(1+adata_pk_a.X[...])
    mat_pk_c = np.log10(1+adata_pk_c.X[...])
    y_a = np.mean(mat_pk_a, axis=1)
    y_c = np.mean(mat_pk_c, axis=1)
    
    
    ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5)# , facecolor='none')
    ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5)# , facecolor='none')
    r_a, p  = stats.spearmanr(x, y_a)
    r_c, p  = stats.spearmanr(x, y_c)

    ax.set_title(f'P{time}\nr_a={r_a:.4f}\nr_c={r_c:.4f}')
    # ax.legend(bbox_to_anchor=(1,1))

    sns.despine(ax=ax)
    ax.grid(False)

axs[0].set_xlabel('RNA PC1 (A-C axis; P17on)')
axs[0].set_ylabel('ATAC')
plt.show()

In [ ]:
fig, axs = plt.subplots(1,7, figsize=(4*7,4)) # , sharex=True, sharey=True)
for cond_order in range(7):
    ax = axs[cond_order]
    time = times[cond_order]
    obssub = adata[adata.obs['cond_order']==cond_order].obs
    
    # get sub
    adatasub = adata[obssub.index]
    adata_pk = adatas_pk[cond_order]
    
    cells_rna = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub.obs.index.values])
    adatasub.obs.index = cells_rna
    cells_atac = adata_pk.obs.index.values
    cells_mult = np.intersect1d(cells_rna, cells_atac) 
    
    x = adatasub[cells_mult].obsm['pca_p17on'][...,0]
    adata_pk_a = adata_pk[:,cond_sig_a[cond_order]][cells_mult]
    adata_pk_c = adata_pk[:,cond_sig_c[cond_order]][cells_mult]
    mat_pk_a = np.log10(1+adata_pk_a.X[...])
    mat_pk_c = np.log10(1+adata_pk_c.X[...])
    y_a = np.mean(mat_pk_a, axis=1)
    y_c = np.mean(mat_pk_c, axis=1)
    
    
    ax.scatter(x, y_a, s=1, color='C0', label='A peaks', alpha=0.5)# , facecolor='none')
    ax.scatter(x, y_c, s=1, color='C2', label='C peaks', alpha=0.5)# , facecolor='none')
    r_a, p  = stats.spearmanr(x, y_a)
    r_c, p  = stats.spearmanr(x, y_c)

    ax.set_title(f'P{time}\nr_a={r_a:.4f}\nr_c={r_c:.4f}')
    # ax.legend(bbox_to_anchor=(1,1))

    sns.despine(ax=ax)
    ax.grid(False)

axs[0].set_xlabel('RNA PC1 (A-C axis; P17on)')
axs[0].set_ylabel('ATAC')
plt.show()